In [1]:
import pandas as pd
DR = pd.read_csv('DR.csv')

def UpdateList(L):
    return L/max(L)

PostTagList = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP1','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBZ','WDT','WP','WP1','WRB']

print DR.columns
print DR.dtypes

for PS in PostTagList:
    print PS
    print max(DR[PS])
    DR[PS] = UpdateList(DR[PS])
    print max(DR[PS])
    print ""
    


Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Comment', u'Duration', u'FDA_Alerts',
       u'Drug_Class', u'Drug_Name', u'UserRating', u'Useful_Reviews', u'Date',
       u'NReviews', u'Drug_Interactions', u'Condition', u'Major', u'Moderate',
       u'Minor', u'DurationCategory', u'CC', u'CD', u'DT', u'EX', u'FW', u'IN',
       u'JJ', u'JJR', u'JJS', u'LS', u'MD', u'NN', u'NNS', u'NNP', u'NNPS',
       u'PDT', u'POS', u'PRP', u'PRP1', u'RB', u'RBR', u'RBS', u'RP', u'TO',
       u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ', u'WDT', u'WP',
       u'WP1', u'WRB', u'Polarity', u'Subjectivity'],
      dtype='object')
Unnamed: 0             int64
Unnamed: 0.1           int64
Comment               object
Duration              object
FDA_Alerts           float64
Drug_Class            object
Drug_Name             object
UserRating           float64
Useful_Reviews        object
Date                  object
NReviews               int64
Drug_Interactions     object
Condition             object
M

In [2]:
#Drop Rows that Have an Error in the 'Useful_Reviews'
inx = []

for i in range(0,len(DR)):
    try:
        k = float(DR.iloc[i,8])
    except:
        inx.append(i)
        
DR = DR.drop(DR.index[inx])

In [3]:
#Drop the first two columns from DR and assign the data frame to DR_Final
DR_Final = DR.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)


In [4]:
print len(DR_Final)
print len(inx)

162732
203


In [5]:
#Coerce 'Useful_Reviews' to float
DR_Final['Useful_Reviews'] = pd.to_numeric(DR_Final['Useful_Reviews'],errors='coerce')

#DR_Sub for Correlation_Matrix
DR_Sub = DR_Final.iloc[:,[2,5,6,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51]]


In [ ]:
# Visualize Correlation_Matrix from DR_Sub
from pandas import set_option
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

set_option('precision',1)
# print DR_New.corr(method='pearson')

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(DR_Sub.corr(),vmin=-1,vmax=1,interpolation='none')
fig.colorbar(cax)
ticks = np.arange(0,44,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
plt.show()

In [ ]:
print DR_Sub.columns
print DR_Sub.dtypes

In [ ]:
# Plot Histograms for each variable attribute in DR_Sub

import matplotlib.pyplot as plt
%matplotlib tk

ax = DR_Sub.iloc[0:len(DR_Sub),41].plot.hist(color='red')
ax.set_xlabel('Polarity',size=20,fontweight='bold')
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
ax.set_ylabel('Counts',size=20,fontweight='bold')
ax.set_title('Distribution of Polarity',fontsize=20,fontweight='bold')


In [ ]:
#Model: Define the Model for 'Major Interactions'

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#To predict 'Major Drug Interactions' Ignore column '11'
DR_Sub_X = DR_Final.iloc[:,[2,5,6,8,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51]]

#Use column '11' to predict 'Major Drug Interactions'
DR_Sub_Y = DR_Final.iloc[:,11]

DRSubXMean = DR_Sub_X.mean()
DRSubYMean = DR_Sub_Y.mean()

seed = 11
validation_size = 0.2

for i in range(0,len(DRSubXMean)):
    DR_Sub_X[DR_Sub_X.columns[i]] = DR_Sub_X[DR_Sub_X.columns[i]].fillna(DRSubXMean[i])
    
DR_Sub_Y = DR_Sub_Y.fillna(DRSubYMean)
    
DR_Sub_X = DR_Sub_X.values
DR_Sub_Y = DR_Sub_Y.values

X_train, X_validation,Y_train,Y_validation = train_test_split(DR_Sub_X,DR_Sub_Y,test_size=validation_size,random_state=seed)

# Drop '11' for 'Major Interactions' model
index = [2,5,6,8,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51]

# Assign the model either 'Major'
modelusecase = [11]

colnames_X = DR_Final.columns[index]
colnames_Y = DR_Final.columns[modelusecase]

X_train = pd.DataFrame(X_train)
X_train.columns = colnames_X
X_train.to_csv('X_train-MajInt.csv')

Y_train = pd.DataFrame(Y_train)
Y_train.columns = colnames_Y
Y_train.to_csv('Y_train-MajInt.csv')

X_validation = pd.DataFrame(X_validation)
X_validation.columns = colnames_X
X_validation.to_csv('X_validation-MajInt.csv')

Y_validation = pd.DataFrame(Y_validation)
Y_validation.columns = colnames_Y
Y_validation.to_csv('Y_validation-MajInt.csv')


In [6]:
import pandas as pd

X_train = pd.read_csv('X_train-MajInt.csv').drop(['Unnamed: 0'],axis=1)
Y_train = pd.read_csv('Y_train-MajInt.csv').drop(['Unnamed: 0'],axis=1)
X_validation = pd.read_csv('X_validation-MajInt.csv').drop(['Unnamed: 0'],axis=1)
Y_validation = pd.read_csv('Y_validation-MajInt.csv').drop(['Unnamed: 0'],axis=1)

print X_train.shape, Y_train.shape
print X_train.columns
print Y_train.columns

print X_train.columns

(130185, 42) (130185, 1)
Index([u'FDA_Alerts', u'UserRating', u'Useful_Reviews', u'NReviews',
       u'Moderate', u'Minor', u'DurationCategory', u'CC', u'CD', u'DT', u'EX',
       u'FW', u'IN', u'JJ', u'JJR', u'JJS', u'LS', u'MD', u'NN', u'NNS',
       u'NNP', u'NNPS', u'PDT', u'POS', u'PRP', u'RB', u'RBR', u'RBS', u'RP',
       u'TO', u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ', u'WDT',
       u'WP', u'WRB', u'Polarity', u'Subjectivity'],
      dtype='object')
Index([u'Major'], dtype='object')
Index([u'FDA_Alerts', u'UserRating', u'Useful_Reviews', u'NReviews',
       u'Moderate', u'Minor', u'DurationCategory', u'CC', u'CD', u'DT', u'EX',
       u'FW', u'IN', u'JJ', u'JJR', u'JJS', u'LS', u'MD', u'NN', u'NNS',
       u'NNP', u'NNPS', u'PDT', u'POS', u'PRP', u'RB', u'RBR', u'RBS', u'RP',
       u'TO', u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ', u'WDT',
       u'WP', u'WRB', u'Polarity', u'Subjectivity'],
      dtype='object')


In [7]:
# Import Statements
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [8]:
#Spotcheck Algorithms
validation_size = 0.2
seed = 11

num_folds = 10
scoring = 'neg_mean_squared_error'

models = []
models.append(('LR1',LinearRegression()))
models.append(('LASSO',Lasso()))
models.append(('EN',ElasticNet()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
# models.append(('SVR',SVR()))

results = []
names = []
print 'ok'
for name,model in models:
#     print name, model
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,Y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(),cv_results.std())
    print msg

print ""
print ""

ok
LR1: -11723.231415 (135.647873)
LASSO: -11786.750272 (142.219988)
EN: -11792.753546 (140.402347)
KNN: -193.600949 (47.808853)
CART: -7.840296 (4.603752)




In [10]:
import matplotlib.pyplot as plt
%matplotlib tk

#Compare Algorithm Responses
fig = plt.figure()
fig.suptitle("Comparison of Algorithms (POS Tag)",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithm',size=20,fontweight='bold')
plt.ylabel('Neg Mean Squared',size=20,fontweight='bold')
plt.show()

In [11]:
#Standardize the Data
pipelines = []

pipelines.append(('ScaledLR',Pipeline([('Scaler',StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO',Pipeline([('Scaler',StandardScaler()),('LASSO',Lasso())])))
pipelines.append(('ScaledEN',Pipeline([('Scaler',StandardScaler()),('EN',ElasticNet())])))
# pipelines.append(('ScaledKNN',Pipeline([('Scaler',StandardScaler()),('KNN',KNeighborsRegressor())])))
pipelines.append(('ScaledCART',Pipeline([('Scaler',StandardScaler()),('CART',DecisionTreeRegressor())])))

results = []
names = []
print 'ok'
for name,model in pipelines:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(model,X_train,Y_train,cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(),cv_results.std())
    print msg
    
print 'ok_end'

ok
ScaledLR: -11723.231415 (135.647873)
ScaledLASSO: -11754.439208 (137.252720)
ScaledEN: -12144.124922 (123.577845)
ScaledCART: -7.764258 (3.303237)
ok_end


In [12]:
#Compare Algorithm Responses
fig = plt.figure()
fig.suptitle("Comparison of Algorithms (POS Tag)",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithm',size=20,fontweight='bold')
plt.ylabel('Neg Mean Squared',size=20,fontweight='bold')
plt.show()

In [13]:
#Ensemble Methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor

ensembles = []
ensembles.append((' ScaledAB',Pipeline([('Scaler',StandardScaler()),('AB',AdaBoostRegressor())])))
# ensembles.append(('ScaledGBM',Pipeline([('Scaler',StandardScaler()),('GBM',GradientBoostingRegressor())])))
ensembles.append(('ScaledRF',Pipeline([('Scaler',StandardScaler()),('RF',RandomForestRegressor())])))
ensembles.append(('ScaledET',Pipeline([('Scaler',StandardScaler()),('ET',ExtraTreesRegressor())])))

seed = 7

results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
print 'ok_end'

/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 ScaledAB: -9711.512055 (739.890836)


/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/krishnaveni/anaconda2/lib/python2.7/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


ScaledRF: -9.173086 (5.084887)
ScaledET: -7.257761 (2.145993)
ok_end


In [ ]:
print len(cv_results)
print results

In [14]:
#Compare Algorithm Responses
fig = plt.figure()
fig.suptitle("Comparison of Algorithms (POS Tag)",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Algorithm',size=20,fontweight='bold')
plt.ylabel('Neg Mean Squared',size=20,fontweight='bold')
plt.show()

In [ ]:
#Fine Tune Extra_Trees
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.model_selection import GridSearchCV

seed = 11
num_folds = 10
scoring = 'neg_mean_squared_error'

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=np.array([10,50,100,500]))
model = ExtraTreesRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds,random_state=seed)
grid = GridSearchCV(estimator=model,param_grid=param_grid,scoring=scoring,cv=kfold)
print 'ok'
grid_result = grid.fit(rescaledX,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means,stds,params):
    print("%f (%f) with: %r" % (mean,stdev,param))


In [ ]:
#Compare Fine-Tuned Responses
import matplotlib.pyplot as plt
%matplotlib tk

fig = plt.figure()
fig.suptitle("Comparison of Trees (Major Interactions)",fontsize=20,fontweight='bold')
ax = fig.add_subplot(111)

names = ['10','50','100','500']
results = [-7.740830, -5.211410, -4.789472, -4.580332]
errorbar = [4.077324, 3.256445, 2.967114, 2.977927]

plt.errorbar(names, results, errorbar, linestyle='--',marker='^',markersize=15,capsize=20)
# ax.set_xticklabels(names)
ax.tick_params(axis='both',labelsize=15)
# ax.set_xticklabels(ax.get_xticks(),fontweight='bold')
# ax.set_yticklabels(ax.get_yticks(),fontweight='bold')
plt.xlabel('Number of Estimators',size=20,fontweight='bold')
plt.ylabel('Neg Mean Squared',size=20,fontweight='bold')
plt.show()

In [ ]:
param_grid
print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

In [ ]:
#Execute the final model: ExtraTrees(n_estimators:100) on the validation set

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

seed = 13
ET = ExtraTreesRegressor(n_estimators=100,random_state=seed)
ET.fit(X_train,Y_train)
predictions = ET.predict(X_validation)
print(mean_squared_error(Y_validation,predictions))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import explained_variance_score

# print(explained_variance_score(Y_validation,predictions))
# print(confusion_matrix(Y_validation,predictions))
# print(classification_report(Y_validation,predictions))
